# Building a Semantic Search Engine to Search for Queries with Transformers

# Semantic Search
Semantic search seeks to improve search accuracy by understanding the content of the search query. In contrast to traditional search engines, that only finds documents based on lexical matches, semantic search can also find synonyms.


## Background
The idea behind semantic search is to embedd all entries in your corpus, which can be sentences, paragraphs, or documents, into a vector space. 

At search time, the query is embedded into the same vector space and the closest embedding from your corpus are found. These entries should have a high semantic overlap with the query.

![SemanticSearch](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/SemanticSearch.png) 


## Similarity Computation

For small corpora (up to about 100k entries) we can compute the cosine-similarity between the query and all entries in the corpus.

For small corpora with few example sentences we compute the embeddings for the corpus as well as for our query.

We then use the [util.pytorch_cos_sim()](../../../docs/usage/semantic_textual_similarity.md) function to compute the cosine similarity between the query and all corpus entries.

For large corpora, sorting all scores would take too much time. Hence, we can use [torch.topk](https://pytorch.org/docs/stable/generated/torch.topk.html) to only get the top k entries.

[Reference](https://github.com/UKPLab/sentence-transformers/tree/master/examples/applications/semantic-search)


## Objective

For today's objective we will create a corpus of around 50000 question titles asked on Quora from an open dataset. Your task will be to compute sentence embeddings and then try to retrieve top 5 similar questions from the corpus for a few example queries mentioned below.

Use [Sentence Transformers](https://github.com/UKPLab/sentence-transformers) which provides a scalable way to generate document embeddings using transformers



## Load Dependencies

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 4.3MB/s 
     |████████████████████████████████| 890kB 17.6MB/s 
     |████████████████████████████████| 3.2MB 25.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=03c242430e750485a0997bf7a8b4cc75819599bc262987aac1fdbda00863874a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 1.2MB 7.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=66b85a202e87b4abdc9bb3a02b3761932791d5dea40beed631ef0b3ec7cf17df
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
Successfully built sentence-transformers


In [3]:
import transformers

In [4]:
import pandas as pd
import numpy as np

## Download and Load Corpus of Questions

In [5]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2021-04-02 17:46:22--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M   105MB/s    in 0.5s    

2021-04-02 17:46:31 (105 MB/s) - ‘quora_duplicate_questions.tsv’ saved [58176133/58176133]



In [6]:
df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t').head(25000)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
corpus = df['question1'].tolist() + df['question2'].tolist()

In [8]:
len(corpus)

50000

## Use Sentence Transformers and Generate Corpus Embeddings

__Hint:__ You can use this tutorial as a reference

[Semantic Search Tutorial](https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/semantic-search/semantic_search.py)


# __Question 1__: Load Pre-trained Embedder Model

Load the __`roberta-large-nli-stsb-mean-tokens`__ model to generate embeddings

In [9]:
from sentence_transformers import SentenceTransformer

In [11]:
model = 'roberta-large-nli-stsb-mean-tokens'
embedder = SentenceTransformer(model)

# __Question 2__: Generate Corpus Embeddings

Generate embeddings for each and every document using the pre-trained model

In [12]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [13]:
corpus_embeddings.shape

torch.Size([50000, 1024])

# __Question 3__: Create a function to print top K similar sentences for a given query

Use cosine similarity by leveraging the pytorch utility in `sentence_transformers` as depicted in the previously linked tutorial.

In [15]:
from sentence_transformers import util
import torch

def print_similar_sentences(query, model_embedder, corpus_embeddings, top_k):
    """
      query: this should be your input query
      model_embedder: this should be your embedding model (pre-trained model which you loaded earlier)
      corpus_embeddings: this should hold the embeddings you generate for your corpus
      top_k: the top k similar queries you should return
    """

    query_embedding = model_embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use torch.topk to find the highest 5 scores
    top_results = torch.topk(cos_scores, k=top_k)
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: %.4f)" % (score))

# __Question 4__: Perform Semantic Search on Sample Questions to get Similar Queries from the Corpus

In [16]:
s = 'What is the step by step guide to invest'
print_similar_sentences(query=s,
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is the step by step guide to invest

Top 5 most similar sentences in corpus:
What is the step by step guide to invest in share market? (Score: 0.8431)
What are the best investment strategy for beginners? (Score: 0.7725)
What are the ways to get an investment for startup? (Score: 0.7692)
How do I invest in stock market? (Score: 0.7558)
How much money will I need to start investing in stock market? (Score: 0.7447)


In [17]:
s = 'What is Data Science?'
print_similar_sentences(query=s, 
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is Data Science?

Top 5 most similar sentences in corpus:
What is data science (Score: 0.9840)
What is actually a data science? (Score: 0.9609)
What does a data scientist do? (Score: 0.8919)
What is big data science? (Score: 0.8633)
What is the difference between data science and data analysis? (Score: 0.7723)


In [18]:
s = 'What is natural language processing?'
print_similar_sentences(query=s, 
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is natural language processing?

Top 5 most similar sentences in corpus:
How does natural language processing work? (Score: 0.9242)
Which are the best schools for studying natural language processing? (Score: 0.6843)
What is the english word for "अंत्योदय"? (Score: 0.6685)
What are natural numbers? (Score: 0.6590)
Who owns Natural Factors? (Score: 0.6589)


In [19]:
s = 'What is natural language processing?'
print_similar_sentences(query=s, 
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is natural language processing?

Top 5 most similar sentences in corpus:
How does natural language processing work? (Score: 0.9242)
Which are the best schools for studying natural language processing? (Score: 0.6843)
What is the english word for "अंत्योदय"? (Score: 0.6685)
What are natural numbers? (Score: 0.6590)
Who owns Natural Factors? (Score: 0.6589)


In [20]:
s = 'Best Harry Potter Movie?'
print_similar_sentences(query=s, 
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: Best Harry Potter Movie?

Top 5 most similar sentences in corpus:
Which Harry Potter movie is the best? (Score: 0.9560)
Which is the best Harry Potter movie? (Score: 0.9456)
Which is your favourite Harry Potter movie and why? (Score: 0.8769)
Where were the Harry Potter movies shot? (Score: 0.8664)
Where was Harry Potter filmed? (Score: 0.8336)


In [21]:
s = 'What is the best smartphone?'
print_similar_sentences(query=s, 
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is the best smartphone?

Top 5 most similar sentences in corpus:
What are the best smartphones? (Score: 0.9829)
What are the best smartphones? (Score: 0.9829)
What is the best smartphone to date? (Score: 0.9759)
What are the best Smartphones tech gadgets? (Score: 0.9262)
Which is the best smartphone to buy now? (Score: 0.9253)


In [22]:
s = 'What is the best starter pokemon?'
print_similar_sentences(query=s, 
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is the best starter pokemon?

Top 5 most similar sentences in corpus:
How do you choose the right starter pokemon in any game? (Score: 0.8680)
What is the best Pokemon GO hack? (Score: 0.7935)
Which set of starter Pokemon would you choose considering all generations and why? (Score: 0.7794)
What are the best Pokemon hacks? (Score: 0.7574)
Which Pokemon evolve with Shiny Stones? (Score: 0.7405)


In [23]:
s = 'Batman or Superman?'
print_similar_sentences(query=s, 
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: Batman or Superman?

Top 5 most similar sentences in corpus:
Why does Batman kill in Batman v Superman? (Score: 0.7654)
What does Batman do? (Score: 0.7581)
Is Batman insane? (Score: 0.7382)
Superheroes: Who would win in a fight between Batman and the Flash? (Score: 0.7381)
Who would win Batman vs Batman? (Score: 0.7156)
